In [1]:
import sys
import os
from pathlib import Path

import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.utils.data_extracting import get_all_account_statement_files, extract_text_from_pdf, get_balance_of_account, get_all_transactions
from src.utils.data_loading import update_google_sheet
from src.utils.data_transforming import extract_transaction_info, add_new_row
from src.utils.google_connection import set_up_google_connection

# Get files

In [2]:
credentials_path = Path('../credentials/cool-plasma-452619-v4-feb20b70d461.json')
downloads_path = Path.home() / 'Downloads'

In [3]:
files = get_all_account_statement_files(downloads_path)
files

[WindowsPath('C:/Users/xaver/Downloads/2115089016_2025_Nr.001_Kontoauszug.pdf'),
 WindowsPath('C:/Users/xaver/Downloads/2115089016_2025_Nr.002_Kontoauszug.pdf'),
 WindowsPath('C:/Users/xaver/Downloads/2115089016_2025_Nr.003_Kontoauszug.pdf'),
 WindowsPath('C:/Users/xaver/Downloads/2115089016_2025_Nr.004_Kontoauszug.pdf'),
 WindowsPath('C:/Users/xaver/Downloads/2115089016_2025_Nr.005_Kontoauszug.pdf'),
 WindowsPath('C:/Users/xaver/Downloads/2115089016_2025_Nr.006_Kontoauszug.pdf'),
 WindowsPath('C:/Users/xaver/Downloads/2115089016_2025_Nr.007_Kontoauszug.pdf'),
 WindowsPath('C:/Users/xaver/Downloads/2115089016_2025_Nr.008_Kontoauszug.pdf')]

In [4]:
for file in files:
    print(file)

C:\Users\xaver\Downloads\2115089016_2025_Nr.001_Kontoauszug.pdf
C:\Users\xaver\Downloads\2115089016_2025_Nr.002_Kontoauszug.pdf
C:\Users\xaver\Downloads\2115089016_2025_Nr.003_Kontoauszug.pdf
C:\Users\xaver\Downloads\2115089016_2025_Nr.004_Kontoauszug.pdf
C:\Users\xaver\Downloads\2115089016_2025_Nr.005_Kontoauszug.pdf
C:\Users\xaver\Downloads\2115089016_2025_Nr.006_Kontoauszug.pdf
C:\Users\xaver\Downloads\2115089016_2025_Nr.007_Kontoauszug.pdf
C:\Users\xaver\Downloads\2115089016_2025_Nr.008_Kontoauszug.pdf


In [20]:
file_path = files[1]
full_pdf_text = extract_text_from_pdf(file_path)
lines = full_pdf_text.split('\n')

In [21]:
acc_balance_old = get_balance_of_account(lines, 'alter Kontostand')
acc_balance_new = get_balance_of_account(lines, 'neuer Kontostand')
print(f'Alter Kontostand, Betrag: {acc_balance_old[0]}€ - Line-Index: {acc_balance_old[1]}')
print(f'Neuer Kontostand, Betrag: {acc_balance_new[0]}€ - Line-Index: {acc_balance_new[1]}')

all_transactions = get_all_transactions(lines, acc_balance_old[1], acc_balance_new[1])
print(f'Anzahl der Transaktionen: {len(all_transactions)}')

Alter Kontostand, Betrag: 4051.93€ - Line-Index: 11
Neuer Kontostand, Betrag: 3275.48€ - Line-Index: 208
[[], ['03.02. 03.02. Überweisungsauftrag 745,00 S', 'Elisabeth Heuser', 'Auto Versicherung SecureGo plus IBAN: DE41500105175409', '388951 BIC: INGDDEFFXXX'], ['03.02. 03.02. Überweisungsauftrag 135,00 S', 'Elisabeth Heuser', 'Geschenk Haie Tickets SecureGo plus IBAN: DE4150010517', '5409388951 BIC: INGDDEFFXXX'], ['04.02. 04.02. Kartenzahlung girocard 5,00 S', 'SODEXO SERVICES DE169201', 'SODEXO SERVICES GMB/BONN/DE', '03.02.2025 um 12:31:12 Uhr 65207252/024454/ECTL/', '37069520/2115089016/1/1225 REF 786576/260047'], ['04.02. 04.02. Überweisungsgutschr. 200,00 H', 'Heide Gudrun Heuser', 'GESCHENK EREF: 03505005035'], ['05.02. 05.02. Basislastschrift 105,00 S', 'Baloise Lebensversic', 'L 2.876.325 105,00 EREF: BALN-DE-LIB-P-0000000595720', '2 MREF: HGS20231994961 CRED: DE81ZZZ00000243242'], ['06.02. 06.02. Basislastschrift 44,00 S', 'PayPal Europe S.a.r.l. et Cie S.C.A', '10400754407

# Google Connection

In [22]:
client, service = set_up_google_connection(credentials_path)

In [23]:
spreadsheet = client.open_by_key("1OnrW1foE-1lOtgfxBv2Y5qqJSDnW4hiYeLpScjgFKxM") # TODO: Put in secrets/env
sheet = spreadsheet.sheet1

sheet_incomes = spreadsheet.worksheet('Einnahmen')
sheet_expenses = spreadsheet.worksheet('Ausgaben')

df_expenses = pd.DataFrame(sheet_expenses.get_all_values())
df_incomes = pd.DataFrame(sheet_incomes.get_all_values())

In [24]:
df_expenses.columns = df_expenses.iloc[0]
df_expenses = df_expenses[1:].reset_index(drop=True)

df_incomes.columns = df_incomes.iloc[0]
df_incomes = df_incomes[1:].reset_index(drop=True)

gsheets = {'Expense': df_expenses, 'Income': df_incomes}

In [25]:
gsheets['Income']

,Wer,Dauerauftrag,Kategorie,01,02,03,04,05,06,07,08,09,10,11,12,Hinweis
0,Jan Leonardo Rütten,Nein,,"36,00",,,,,,,,,,,,
1,Deutsche Telekom AG,Ja,,"1141,25",,,,,,,,,,,,
2,VR-BANK BONN RHEIN-SIEG,Nein,,"955,00",,,,,,,,,,,,
3,Renten Service,Ja,,"22,92",,,,,,,,,,,,
4,Deutsche Post AG,Ja,,"305,55",,,,,,,,,,,,


In [26]:
gsheets['Expense']

,Wer,Dauerauftrag,Kategorie,01,02,03,04,05,06,07,08,09,10,11,12,Hinweis
0,Elisabeth Heuser,Nein,,"76,00",,,,,,,,,,,,
1,Freundeskreis Elisabeth-Hospiz e.V.,Ja,,"10,00",,,,,,,,,,,,
2,Baloise Lebensversic,Ja,,"105,00",,,,,,,,,,,,
3,Annabelle Schulz,Ja,,"17,60",,,,,,,,,,,,
4,PayPal Europe S.a.r.l. et Cie S.C.A,Nein,,"6,11",,,,,,,,,,,,
5,PayPal Europe S.a.r.l. et Cie S.C.A,Nein,,"5,99",,,,,,,,,,,,
6,Adyen N.V.,Nein,,"31,50",,,,,,,,,,,,
7,PayPal Europe S.a.r.l. et Cie S.C.A,Nein,,"108,00",,,,,,,,,,,,
8,REWE Alexander Ker,Nein,,"3,52",,,,,,,,,,,,
9,REWE Ali Sahin oHG,Nein,,"49,49",,,,,,,,,,,,


In [27]:
for transaction_index in range(len(all_transactions)):
    transaction = all_transactions[transaction_index]

    transaction_type, df, name, transaction_value, month = (
        extract_transaction_info(transaction, gsheets)
    )

    add_new_row(
        df,
        name,
        month,
        transaction_value,
        transaction_type,
        gsheets,
        general_account=True,
    )

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


In [28]:
gsheets['Income']

,Wer,Dauerauftrag,Kategorie,01,02,03,04,05,06,07,08,09,10,11,12,Hinweis
0,Jan Leonardo Rütten,Nein,,"36,00",,,,,,,,,,,,
1,Deutsche Telekom AG,Ja,,"1141,25",,,,,,,,,,,,
2,VR-BANK BONN RHEIN-SIEG,Nein,,"955,00",,,,,,,,,,,,
3,Renten Service,Ja,,"22,92",,,,,,,,,,,,
4,Deutsche Post AG,Ja,,"305,55",,,,,,,,,,,,
5,Heide Gudrun Heuser,None,None,None,200.0,None,None,None,None,None,None,None,None,None,None,None
6,Deutsche Telekom AG,None,None,None,1133.25,None,None,None,None,None,None,None,None,None,None,None
7,Renten Service,None,None,None,22.92,None,None,None,None,None,None,None,None,None,None,None
8,Elisabeth Heuser,None,None,None,85.0,None,None,None,None,None,None,None,None,None,None,None
9,Matthias Lade,None,None,None,10000.0,None,None,None,None,None,None,None,None,None,None,None


In [29]:
gsheets['Expense']

,Wer,Dauerauftrag,Kategorie,01,02,03,04,05,06,07,08,09,10,11,12,Hinweis
0,Elisabeth Heuser,Nein,,"76,00",,,,,,,,,,,,
1,Freundeskreis Elisabeth-Hospiz e.V.,Ja,,"10,00",,,,,,,,,,,,
2,Baloise Lebensversic,Ja,,"105,00",,,,,,,,,,,,
3,Annabelle Schulz,Ja,,"17,60",,,,,,,,,,,,
4,PayPal Europe S.a.r.l. et Cie S.C.A,Nein,,"6,11",,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,1224-30105 FEUCHT,None,None,None,1.0,None,None,None,None,None,None,None,None,None,None,None
64,Adyen N.V.,None,None,None,12.2,None,None,None,None,None,None,None,None,None,None,None
65,Xaver Heuser,None,None,None,5000.0,None,None,None,None,None,None,None,None,None,None,None
66,EVENT FOOD SCHROEDER ARMINIA,None,None,None,6.6,None,None,None,None,None,None,None,None,None,None,None


# Load data into Google Sheet

In [30]:
update_google_sheet(sheet_expenses, gsheets['Expense'])
update_google_sheet(sheet_incomes, gsheets['Income'])
print('✅ All changes saved to Google Sheets!')

✅ All changes saved to Google Sheets!
